In [1]:
pip install opencv-contrib-python

    100% |████████████████████████████████| 34.2MB 48kB/s  eta 0:00:011   20% |██████▋                         | 7.1MB 58.3MB/s eta 0:00:01    27% |████████▊                       | 9.4MB 32.8MB/s eta 0:00:01
    100% |████████████████████████████████| 20.2MB 80kB/s  eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [12]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, AspectAwarePreprocessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
##
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from imutils import paths
import numpy as np
import os

In [13]:
args = {
    'dataset': r'/home/mhasan3/Desktop/WorkFolder/flowers17/images/',
    'model': 'flowers17.model'
}

In [14]:
# construct the image generator
aug = ImageDataGenerator(rotation_range=30,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,
                         fill_mode='nearest')

In [15]:
# grab the list of images that we'll be describing then extract the class 
# label names from the imagePaths
imagePaths = list(paths.list_images(args['dataset']))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [16]:
# initialise the image preprocessor
aap = AspectAwarePreprocessor(224,224)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixels
sdl = SimpleDatasetLoader(preprocessors=[aap, iap])
data, labels = sdl.load(imagePaths, verbose=500)
data = data.astype('float') / 255.0

[INFO] processed 500/1360
[INFO] processed 1000/1360


In [17]:
# partition the data into training and test splits
trainx, testx, trainy, testy = train_test_split(data,
                                                labels,
                                                test_size=0.25,
                                                random_state=42)

# convert labels from integers into vectors
le = LabelBinarizer()
trainy = le.fit_transform(trainy)
testy = le.transform(testy)

In [18]:
# load the VGG16 network ensuring the head FC layer sets are left off
baseModel = VGG16(weights='imagenet', include_top=False, 
                  input_tensor=Input(shape=(224, 224, 3))) # type: Model

# initialise the new head of the network, a set of FC layers followed by 
# softmax
headModel = FCHeadNet.builld(baseModel, len(classNames), D=256) # type: Model

# place the head FC model on top of the base model - this will be the actual
# model to train
model = Model(inputs=baseModel.input, outputs=headModel)

In [19]:
# loop over all the layers in the base model and freeze them
for layer in baseModel.layers:
    layer.trainable = False

In [20]:
# compile and optimise model
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# train the head for few epochs
model.fit_generator(aug.flow(trainx, trainy, batch_size=32),
                    validation_data=(testx, testy),
                    epochs=25,
                    steps_per_epoch=len(trainx) // 32,
                    verbose=1)

Epoch 1/25
32/32 [==============================] - 33s 1s/step - loss: 6.5716 - acc: 0.1588 - val_loss: 2.2510 - val_acc: 0.3118
Epoch 2/25
32/32 [==============================] - 12s 376ms/step - loss: 2.2449 - acc: 0.3029 - val_loss: 1.7157 - val_acc: 0.4412
Epoch 3/25
32/32 [==============================] - 12s 382ms/step - loss: 1.9059 - acc: 0.4176 - val_loss: 1.3440 - val_acc: 0.6029
Epoch 4/25
32/32 [==============================] - 12s 375ms/step - loss: 1.6611 - acc: 0.4618 - val_loss: 0.9840 - val_acc: 0.7559
Epoch 5/25
32/32 [==============================] - 12s 376ms/step - loss: 1.4388 - acc: 0.5451 - val_loss: 0.8531 - val_acc: 0.7471
Epoch 6/25
32/32 [==============================] - 12s 376ms/step - loss: 1.3181 - acc: 0.5686 - val_loss: 0.8230 - val_acc: 0.7265
Epoch 7/25
32/32 [==============================] - 13s 399ms/step - loss: 1.2272 - acc: 0.6020 - val_loss: 0.7702 - val_acc: 0.7559
Epoch 8/25
32/32 [==============================] - 12s 377ms/step - los

In [21]:
# evaluate the network
preds = model.predict(testx, batch_size=32)
print(classification_report(testy.argmax(axis=1),
                            preds.argmax(axis=1),
                            target_names=classNames))

              precision    recall  f1-score   support

    bluebell       0.63      0.95      0.76        20
   buttercup       0.83      0.91      0.87        22
   coltsfoot       0.92      0.73      0.81        15
     cowslip       0.68      0.85      0.76        20
      crocus       1.00      0.70      0.82        23
    daffodil       0.70      0.89      0.78        18
       daisy       1.00      0.89      0.94        19
   dandelion       0.90      0.90      0.90        20
  fritillary       1.00      0.75      0.86        20
        iris       0.88      0.94      0.91        16
  lilyvalley       0.94      0.74      0.83        23
       pansy       1.00      0.88      0.93        16
    snowdrop       0.72      0.91      0.81        23
   sunflower       0.95      1.00      0.97        19
   tigerlily       0.96      0.93      0.94        27
       tulip       0.73      0.52      0.61        21
  windflower       0.89      0.94      0.92        18

    accuracy              

In [22]:
# now that FC layers have been trained, lets unfreeze the final set of CONV 
# layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True

In [24]:
# for changes to take effect we need to recompile model
opt = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.fit_generator(aug.flow(trainx, trainy, batch_size=32),
                    validation_data=(testx, testy),
                    epochs=100,
                    steps_per_epoch=len(trainx) // 32,
                    verbose=1)

Epoch 1/100
32/32 [==============================] - 14s 428ms/step - loss: 0.4800 - acc: 0.8333 - val_loss: 0.3780 - val_acc: 0.8941
Epoch 2/100
32/32 [==============================] - 14s 425ms/step - loss: 0.3705 - acc: 0.8843 - val_loss: 0.3522 - val_acc: 0.8971
Epoch 3/100
32/32 [==============================] - 13s 407ms/step - loss: 0.3569 - acc: 0.8892 - val_loss: 0.3311 - val_acc: 0.9118
Epoch 4/100
32/32 [==============================] - 13s 405ms/step - loss: 0.3498 - acc: 0.8824 - val_loss: 0.2959 - val_acc: 0.9235
Epoch 5/100
32/32 [==============================] - 13s 412ms/step - loss: 0.3274 - acc: 0.8990 - val_loss: 0.3419 - val_acc: 0.9147
Epoch 6/100
32/32 [==============================] - 13s 408ms/step - loss: 0.3314 - acc: 0.8892 - val_loss: 0.3117 - val_acc: 0.9206
Epoch 7/100
32/32 [==============================] - 13s 410ms/step - loss: 0.2759 - acc: 0.9078 - val_loss: 0.3091 - val_acc: 0.9088
Epoch 8/100
32/32 [==============================] - 13s 411ms

32/32 [==============================] - 13s 410ms/step - loss: 0.1798 - acc: 0.9314 - val_loss: 0.2758 - val_acc: 0.9294
Epoch 39/100
32/32 [==============================] - 13s 405ms/step - loss: 0.1723 - acc: 0.9451 - val_loss: 0.2607 - val_acc: 0.9382
Epoch 40/100
32/32 [==============================] - 13s 408ms/step - loss: 0.1661 - acc: 0.9422 - val_loss: 0.2974 - val_acc: 0.9324
Epoch 41/100
32/32 [==============================] - 13s 415ms/step - loss: 0.1930 - acc: 0.9333 - val_loss: 0.2773 - val_acc: 0.9324
Epoch 42/100
32/32 [==============================] - 13s 407ms/step - loss: 0.1726 - acc: 0.9402 - val_loss: 0.3102 - val_acc: 0.9294
Epoch 43/100
32/32 [==============================] - 13s 406ms/step - loss: 0.1773 - acc: 0.9314 - val_loss: 0.2691 - val_acc: 0.9353
Epoch 44/100
32/32 [==============================] - 13s 409ms/step - loss: 0.1582 - acc: 0.9500 - val_loss: 0.3068 - val_acc: 0.9235
Epoch 45/100
32/32 [==============================] - 13s 406ms/step

32/32 [==============================] - 13s 406ms/step - loss: 0.1189 - acc: 0.9588 - val_loss: 0.2469 - val_acc: 0.9412
Epoch 76/100
32/32 [==============================] - 14s 426ms/step - loss: 0.1428 - acc: 0.9569 - val_loss: 0.2501 - val_acc: 0.9441
Epoch 77/100
32/32 [==============================] - 14s 440ms/step - loss: 0.1234 - acc: 0.9520 - val_loss: 0.2685 - val_acc: 0.9353
Epoch 78/100
32/32 [==============================] - 13s 421ms/step - loss: 0.1181 - acc: 0.9520 - val_loss: 0.2820 - val_acc: 0.9353
Epoch 79/100
32/32 [==============================] - 14s 432ms/step - loss: 0.1165 - acc: 0.9627 - val_loss: 0.2841 - val_acc: 0.9412
Epoch 80/100
32/32 [==============================] - 13s 409ms/step - loss: 0.1424 - acc: 0.9490 - val_loss: 0.2501 - val_acc: 0.9382
Epoch 81/100
32/32 [==============================] - 13s 408ms/step - loss: 0.1459 - acc: 0.9490 - val_loss: 0.2737 - val_acc: 0.9382
Epoch 82/100
32/32 [==============================] - 13s 406ms/step

In [25]:
# evaluate the network again
preds = model.predict(testx, batch_size=32)
print(classification_report(testy.argmax(axis=1),
                            preds.argmax(axis=1),
                            target_names=classNames))

              precision    recall  f1-score   support

    bluebell       0.95      0.95      0.95        20
   buttercup       1.00      0.91      0.95        22
   coltsfoot       0.88      1.00      0.94        15
     cowslip       0.74      1.00      0.85        20
      crocus       1.00      0.96      0.98        23
    daffodil       1.00      0.83      0.91        18
       daisy       1.00      0.95      0.97        19
   dandelion       1.00      0.90      0.95        20
  fritillary       0.95      0.90      0.92        20
        iris       1.00      0.81      0.90        16
  lilyvalley       0.88      1.00      0.94        23
       pansy       0.94      1.00      0.97        16
    snowdrop       0.88      1.00      0.94        23
   sunflower       1.00      1.00      1.00        19
   tigerlily       0.96      0.96      0.96        27
       tulip       1.00      0.86      0.92        21
  windflower       0.94      0.94      0.94        18

    accuracy              

In [26]:
# save serialised model
model.save(args['model'])